In [1]:
#https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e

In [2]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
#Header passed to mimic an actual browser
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [4]:
site = 'https://casa.sapo.pt'
pagina = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&lp=10000&or=10'

response = get(pagina, headers=headers)

response, response.text


(<Response [200]>,
 '\r\n<!DOCTYPE html>\r\n\r\n<html lang="pt-PT">\r\n<head><title>\r\n\tCasas para Venda, Apartamentos Com Ofertas recentes, de 10\xa0000 € no Distrito de Lisboa, CASA SAPO - Portal Nacional de Imobiliário\r\n</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />\r\n    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="06/06/2019 17:12" />\r\n    \r\n<meta name="content-language" content="pt-PT" />\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0" />\r\n<meta name="description" content="Casas para Venda, Apartamentos Com Ofertas recentes, de 10\xa0000 € no Distrito de Lisboa, Casas para Venda, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />\r\n<meta name="keywords" content="Casas para Venda, Apartamentos Com Ofertas 

In [5]:
html = BeautifulSoup(response.text, 'html.parser')

In [6]:
houseContainers = html.find_all('div', class_="searchResultProperty")

In [19]:
first = houseContainers[5]
first

<div class="searchResultProperty" data-uid="69a63a68-8873-11e9-8325-060000000052">
<div class="photoContainer">
<img alt="Ver Apartamento T3 com garagem, Lumiar em Lisboa" data-error="/Themes/Global/Images/NoPic280x210.jpg" data-src="https://media.casasapo.pt/Z320x240/Wnone/S5/C6416/P11803859/Tphoto/IDd31cb400-0000-0500-0000-00000627d215.jpg" data-src-retina="https://media.casasapo.pt/Z640x480/Wnone/S5/C6416/P11803859/Tphoto/IDd31cb400-0000-0500-0000-00000627d215.jpg" id="MC_PropertyInList_repProperties_thumbImage_5" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7"/>
<a class="photoLayer" href="/comprar-apartamento-t3-lisboa-lumiar-alta-de-lisboa-perto-centro-da-cidade,transportes-publicos,escola-com-garagem-estacionamento-tem-jardim,cozinha-equipada,escolas,aquecimento-central-69a63a68-8873-11e9-8325-060000000052.html?fpn=1" id="MC_PropertyInList_repProperties_hlRealestateDetail_Image_5" title="Ver Apartamento T3 com garagem, Lumiar em Lisboa"></a>


In [8]:
#Get values for the first house.
first.find_all('span')


[<span>
                         Apartamento T2+1, Penha de França (Anjos), Arroios, Lisboa
                     </span>,
 <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('d29dfbbd-e5ae-4416-ac8a-37c14cc75f4c', '13', '1', true); return false;" style="z-index: 9999;">Contacte Anunciante</span>,
 <span>289 000 <strong title="Euro">€</strong></span>]

In [9]:
#Get the price
var_1=first.find_all('span')[2].text
var_1

'289\xa0000 €'

In [10]:
var_1 = var_1.replace('\xa0','')
var_1

'289000 €'

In [11]:
var_1 = int(''.join(itertools.takewhile(str.isdigit,var_1)))
var_1, type(var_1)

(289000, int)

In [20]:
#GetLocation
location = first.find_all('p', class_='searchPropertyLocation')[0].text
location

'\r\n                    Alta de Lisboa, Lumiar, Lisboa\r\n                '

In [13]:
location = location[0].text.replace("\r\n",'')
location

'                    Ajuda, Lisboa                '

In [21]:
import re #RegEx module for manipulation strings
locationX = re.sub('\s', ' ',location)
locationX = locationX.split(',')
locationX

['                      Alta de Lisboa',
 ' Lumiar',
 ' Lisboa                  ']

In [22]:
#Get ap Size
size = first.find_all('p')[7].text
size

'141m²'

In [23]:
sizeX = (''.join(itertools.takewhile(str.isdigit,size)))
sizeX

'141'

In [25]:
#Get Date the ap was posted
pd.to_datetime(first.find_all('div', class_='searchPropertyDate'))


DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [26]:
#Get description
description = first.find_all('p', class_='searchPropertyDescription')[0].text
description


'\r\n                    A pensar em si, este Condomínio foi caracterizado para usufruir do melhor que Lisboa tem, entre o jardim privado aos materiais de elevada qualidade, trazem a este espaço, a paz, o sossego, como também o privilégio de viver (...)\r\n                '

In [19]:
description = description.replace('\r\n','')        
description

'                    Apartamento com vista soberba sobre o sul e o rio Tejo.  obra recentemente acabada. Sala com cozinha americana equipada com optimos electrodomésticos, caixilhos abertura total topo de gama, casa de banho em pedra. Dois (...)                '

In [29]:
#Get the links
url = ''
for url in first.find_all('a'):
    #url.append(url.get('href'))
    #print(url.get('href'))
    url = url.get('href')
url

'/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151'

In [31]:
urlx = site + url
urlx

'https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151'

In [40]:
for url in first.find_all('a'):
    print(url.get('href'))

/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151
/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151
/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151
/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151


In [39]:
a

'/comprar-alugar-apartamento-t2-lisboa-ajuda-tem-arrecadacao,estacionamento-c5d06008-0a42-4e21-9bf1-52de91db7098.html?fpn=1&g3pid=561151'